In [ ]:
import os

PRESET = os.environ.setdefault("PRESET", "spu_lr")


In [ ]:
import asyncio
from opentelemetry import trace
from opentelemetry.sdk.environment_variables import OTEL_SERVICE_NAME

from secretnote.instrumentation.sdk import (
    setup_tracing,
    setup_debug_exporter,
    dump_tracing_context,
)

os.environ[OTEL_SERVICE_NAME] = f"{PRESET}:driver"
setup_tracing()
setup_debug_exporter()

tracer = trace.get_tracer(__name__)

with tracer.start_as_current_span("caller"):
    tracing_context = dump_tracing_context()

    p1, p2 = await asyncio.gather(
        asyncio.create_subprocess_exec(
            "jupyter",
            "execute",
            "sim.ipynb",
            env={
                **os.environ,
                **tracing_context,
                "SELF_PARTY": "alice",
                "RAY_ADDRESS": "127.0.0.1:32400",
            },
        ),
        asyncio.create_subprocess_exec(
            "jupyter",
            "execute",
            "sim.ipynb",
            env={
                **os.environ,
                **tracing_context,
                "SELF_PARTY": "bob",
                "RAY_ADDRESS": "127.0.0.1:32401",
            },
        ),
    )

    await asyncio.gather(p1.wait(), p2.wait())
